In [1]:
import azure.core
from azure.ai.ml.entities import AmlCompute
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ClientSecretCredential
import os



In [2]:
subscription_id=os.getenv('SUBSCRIPTION_ID')
resource_group_name=os.getenv('RESOURCE_GROUP_NAME')
workspace_name=os.getenv('WORKSPACE_NAME')

In [5]:
client_id=os.getenv('AZURE_CLIENT_ID')
client_secret=os.getenv('AZURE_CLIENT_SECRET')
tenant_id=os.getenv('AZURE_TENANT_ID')

In [6]:
credentials=ClientSecretCredential(client_id=client_id, client_secret=client_secret, tenant_id=tenant_id)

In [7]:
from azure.ai.ml.entities import Workspace


ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group_name,
    workspace_name
)


In [14]:
#Cluster name and creat it - if not exists
cpu_compute_target_name = "cpu-cluster00"

try:
    
    cpu_compute_target = ml_client.compute.get(cpu_compute_target_name)
    print(f"This Compute Target: {cpu_compute_target_name} already exists, we will reuse it")
except Exception:
    print(f"Creating a new compute target: {cpu_compute_target_name}")
    
    cpu_cluster= AmlCompute(
       name="cpu-cluster00",
       type="amlcompute",
       size="STANDARD_DS3_V2",
       min_instances=0,
       max_instances=1,
       idle_time_before_scale_down=240,
       location="East US")
    cpu_cluster=ml_client.begin_create_or_update(cpu_cluster)
 
   
       

This Compute Target: cpu-cluster00 already exists, we will reuse it


In [9]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Environment

In [33]:
file_model = Model(
    path="best_knn_model.pkl",
    type=AssetTypes.CUSTOM_MODEL,
    name="Attrition_knn_model",
    description="Model created from local jupyter lab.",
)
ml_client.models.create_or_update(file_model)

Uploading best_knn_model.pkl (< 1 MB): 100%|###########################| 3.77M/3.77M [00:11<00:00, 339kB/s]




Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'Attrition_knn_model', 'description': 'Model created from local jupyter lab.', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/7c109b04-bd25-4d9a-9983-ddd90b981639/resourceGroups/ML_Attrition_Deployment/providers/Microsoft.MachineLearningServices/workspaces/Attrition_Deployment/models/Attrition_knn_model/versions/2', 'Resource__source_path': None, 'base_path': 'C:\\Users\\jakub\\Desktop\\programovani\\Attrition_Analysis', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000010CFC35B9D0>, 'serialize': <msrest.serialization.Serializer object at 0x0000010CFC25E010>, 'version': '2', 'latest_version': None, 'path': 'azureml://subscriptions/7c109b04-bd25-4d9a-9983-ddd90b981639/resourceGroups/ML_Attrition_Deployment/workspaces/Attrition_Deployment/datastores/workspaceblobstore/paths/LocalUp

In [10]:
models = ml_client.models.list()
for model in models:
    print(model.name)

Attrition_knn_model


In [10]:
custom_env_name = "attrit_env"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Attrition classif pipeline",
    conda_file="envir.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="0.1.1",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name attrit_env is registered to workspace, the environment version is 0.1.1


### Create a score.py for evaluate new instances

In [12]:
%%writefile score.py
def init():
    global model, preprocessor
    model_path = Model.get_model_path(model_name='best_knn_model')  
    model = joblib.load(model_path)
    preprocessor = joblib.load('preprocessor.pkl')  

# Perform scoring using the loaded model and preprocessor
def run(raw_data):
    try:
        data = json.loads(raw_data)
       

        # Extract features from the input data
        features = data['features']

        # Apply the preprocessing pipeline to the input data
        preprocessed_data = preprocessor.transform(features)

        # Use the loaded model for predictions
        predictions = model.predict(preprocessed_data)

        # Return the predictions as a JSON object
        return json.dumps({"predictions": predictions.tolist()})
    except Exception as e:
        error = str(e)
        return json.dumps({"error": error})

Writing score.py
